In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.datasets import fetch_mldata, load_svmlight_file
from sklearn.model_selection import train_test_split

In [17]:
import warnings
warnings.filterwarnings('ignore')
from tensorflow.examples.tutorials.mnist import input_data
minist_data = input_data.read_data_sets('/tmp/data/')
X_train=minist_data.train.images
y_train=minist_data.train.labels.astype('int')
X_test=minist_data.test.images
y_test=minist_data.test.labels.astype('int')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [42]:
from functools import partial
learning_rate=0.001
n_h1=100
n_h2=100
n_h3=100
n_h4=100
n_h5=100
n_o=10
epoch=30
batch=64
loop=X_train.shape[0]//batch+1
best_test_acc=0
try_times=5
stop=False

def shuffle_data(X,y):
    index = np.random.permutation(X.shape[0])
    return X[index],y[index]

def get_batch(X,y,index,batch):
    end = (index+1)*batch if (index+1)*batch < X.shape[0] else X.shape[0]-1
    return X[index*batch:end],y[index*batch:end]


tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=[None,X_train.shape[1]],name='X')
y=tf.placeholder(tf.int32,shape=[None,],name='y')

layer=partial(tf.layers.dense,activation=tf.nn.elu,kernel_initializer=tf.variance_scaling_initializer(mode='fan_avg'))
with tf.name_scope('dnn'):
    h1=layer(X,n_h1,name='h1')
    h2=layer(h1,n_h2,name='h2')
    h3=layer(h2,n_h3,name='h3')
    h4=layer(h3,n_h4,name='h4')
    h5=layer(h4,n_h5,name='h5')    
    logits=tf.layers.dense(h5,n_o,name='logits')

with tf.name_scope('train'):
    xentropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits))
    loss=tf.reduce_mean(xentropy)
    
    training=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope('eval'):
    b=tf.nn.in_top_k(logits,y,1,name='in-top-k')
    accuracy=tf.reduce_mean(tf.cast(b,tf.float32),name='eva')


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    n_try=0
    for e in range(epoch):
        X_tmp,y_tmp=shuffle_data(X_train,y_train)
        for l in range(loop):
            X_batch,y_batch=get_batch(X_tmp,y_tmp,l,batch)
            sess.run(training,feed_dict={X:X_batch,y:y_batch})
        test_acc=accuracy.eval({X:minist_data.test.images,y:minist_data.test.labels})
        print('loss:%f,acc_train:%f,acc_test:%f'%(loss.eval({X:X_batch,y:y_batch}),accuracy.eval({X:X_batch,y:y_batch}),test_acc))
        if test_acc>best_test_acc:
            best_test_acc=test_acc
            n_try=0
        elif n_try<try_times:
            n_try+=1
        else:
            stop=True
            print(best_test_acc)
            break                   


loss:0.450090,acc_train:0.913043,acc_test:0.946100
loss:0.024185,acc_train:1.000000,acc_test:0.960400
loss:0.130232,acc_train:0.956522,acc_test:0.965200
loss:0.004517,acc_train:1.000000,acc_test:0.969600
loss:0.033703,acc_train:1.000000,acc_test:0.974000
loss:0.005785,acc_train:1.000000,acc_test:0.972000
loss:0.013416,acc_train:1.000000,acc_test:0.974100
loss:0.067975,acc_train:1.000000,acc_test:0.976900
loss:0.003312,acc_train:1.000000,acc_test:0.971500
loss:0.010555,acc_train:1.000000,acc_test:0.975400
loss:0.003648,acc_train:1.000000,acc_test:0.975000
loss:0.000097,acc_train:1.000000,acc_test:0.978500
loss:0.006828,acc_train:1.000000,acc_test:0.977000
loss:0.001430,acc_train:1.000000,acc_test:0.974200
loss:0.032663,acc_train:1.000000,acc_test:0.975800
loss:0.004330,acc_train:1.000000,acc_test:0.976700
loss:0.005661,acc_train:1.000000,acc_test:0.976000
loss:0.000060,acc_train:1.000000,acc_test:0.973300
0.9785


In [24]:
data=fetch_california_housing()
X,y=data.data,data.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=42)
sc=StandardScaler()
X_train_s=sc.fit_transform(X_train)
X_test_s=sc.transform(X_test)

In [26]:
learning_rate=0.1
momentum=0.9
n_features=X_train.shape[1]
n_data=X_train.shape[0]
n_h1=10
n_h2=10
n_h3=1
epoch=20
batch=64

tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=[None,n_features],name='X')
y=tf.placeholder(tf.float32,shape=[None,],name='y')

initializer = tf.variance_scaling_initializer(mode='fan_avg')
h1=tf.layers.dense(X,n_h1,kernel_initializer=initializer,activation=tf.nn.elu,name='h1')
h2=tf.layers.dense(h1,n_h2,kernel_initializer=initializer,activation=tf.nn.elu,name='h2')
h3=tf.layers.dense(h2,n_h3,kernel_initializer=initializer,name='h3')


with tf.name_scope('train'):
    err=y-h3
    loss=tf.reduce_mean(tf.square(err),name='mse')
    opt=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=momentum).minimize(loss)

# with tf.name_scope('eval'):

def shuffle_data(X,y):
    index = np.random.permutation(n_data)
    return X[index],y[index]

def get_batch(X,y,index,batch):
    return X[index*batch:(index+1)*batch],y[index*batch:(index+1)*batch]

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for e in range(epoch):
        n_round=n_data//batch
        X_tmp,y_tmp=shuffle_data(X_train_s,y_train)
        for n in range(n_round):
            X_batch,y_batch=get_batch(X_tmp,y_tmp,n,batch)
            sess.run(opt,feed_dict={X:X_batch,y:y_batch})
        print(loss.eval(feed_dict={X:X_test_s,y:y_test}))

1.3310632
1.3253576
1.3074894
1.3116977
1.3038616
1.3139673
1.3089169
1.3307462
1.3106983
1.3171688
1.3058139
1.4366393
1.3398427
1.3126721
1.3133603
1.3120388
1.3215576
1.3064449
1.3224119
1.3476644
